## 1. Data Exploration (Lena)

1. Calculate the total number of customers in each section

2. Calculate the total number of customers in each section over time

3. Display the number of customers at checkout over time

4. Calculate the time each customer spent in the market

5. Calculate the total number of customers in the supermarket over time.

In [1]:
import pandas as pd

In [2]:
# Read the data
monday = pd.read_csv('../data/monday.csv', sep=';')
tuesday = monday = pd.read_csv('../data/tuesday.csv', sep=';')
wednesday =  pd.read_csv('../data/wednesday.csv', sep=';')
thursday = pd.read_csv('../data/thursday.csv', sep=';')
friday = pd.read_csv('../data/friday.csv', sep=';')

In [61]:
# 1. Calculate the total number of customers in each section
monday_customers = monday.groupby('location').count()
tuesday_customers = tuesday.groupby('location').count()
wednesday_customers = wednesday.groupby('location').count()
thursday_customers = thursday.groupby('location').count()
friday_customers = friday.groupby('location').count()

print(monday_customers)
print(tuesday_customers)
print(wednesday_customers)
print(thursday_customers)
print(friday_customers)

          timestamp  customer_no
location                        
checkout       1437         1437
dairy           895          895
drinks          797          797
fruit          1005         1005
spices          750          750
          timestamp  customer_no
location                        
checkout       1420         1420
dairy           911          911
drinks          713          713
fruit           976          976
spices          694          694
          timestamp  customer_no
location                        
checkout       1526         1526
dairy           976          976
drinks          775          775
fruit          1043         1043
spices          730          730
          timestamp  customer_no
location                        
checkout       1532         1532
dairy           965          965
drinks          776          776
fruit          1041         1041
spices          790          790
          timestamp  customer_no
location                        
checkout  

In [63]:
# 2. Calculate the total number of customers in each section over time
total = monday_customers + tuesday_customers + wednesday_customers + thursday_customers + friday_customers
total

,timestamp,customer_no
location,,
checkout,7417,7417
dairy,4679,4679
drinks,3905,3905
fruit,5122,5122
spices,3754,3754
